In [1]:
library(Boruta)
library(dplyr)
library(readr)
library(randomForest)
library(pROC)

Warning message:
"package 'Boruta' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


Warning message:
"package 'pROC' was built under R version 4.4.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




In [2]:
##Read CSV file
whole_data <- read.csv("Soil_MetaFeature_Data.csv")

#Subset DataFrame to include only a specific genus at a time, replace as needed.
subset_data <- whole_data[whole_data$`FUNGI_Taxonomy` == 'Chateomium', ]

#Prints subset_data to verify what we're grabbing.
subset_data
dim(subset_data)

sampleid           Filename                Batch Type   InjOrder plate
1   p322_blue_C10_218  p322_blue_C10_218.mzML  NA    sample 218      p322 
2   p322_blue_C11_69   p322_blue_C11_69.mzML   NA    sample 69       p322 
3   p322_blue_C12_296  p322_blue_C12_296.mzML  NA    sample 296      p322 
4   p322_blue_C5_118   p322_blue_C5_118.mzML   NA    sample 118      p322 
5   p322_blue_C6_242   p322_blue_C6_242.mzML   NA    sample 242      p322 
6   p322_blue_C7_194   p322_blue_C7_194.mzML   NA    sample 194      p322 
7   p322_blue_C8_310   p322_blue_C8_310.mzML   NA    sample 310      p322 
8   p322_blue_C9_190   p322_blue_C9_190.mzML   NA    sample 190      p322 
9   p322_blue_D10_67   p322_blue_D10_67.mzML   NA    sample 67       p322 
10  p322_blue_D11_308  p322_blue_D11_308.mzML  NA    sample 308      p322 
11  p322_blue_D12_264  p322_blue_D12_264.mzML  NA    sample 264      p322 
12  p322_blue_D1_261   p322_blue_D1_261.mzML   NA    sample 261      p322 
13  p322_blue_D2_20    p322_blue_D2_20.mzML    NA    sample 20       p322 
14  p322_blue_D3_160   p322_blue_D3_160.mzML   NA    sample 160      p322 
15  p322_blue_D4_313   p322_blue_D4_313.mzML   NA    sample 313      p322 
16  p322_blue_D5_134   p322_blue_D5_134.mzML   NA    sample 134      p322 
17  p322_blue_D6_89    p322_blue_D6_89.mzML    NA    sample 89       p322 
18  p322_blue_D7_90    p322_blue_D7_90.mzML    NA    sample 90       p322 
19  p322_blue_D8_170   p322_blue_D8_170.mzML   NA    sample 170      p322 
20  p322_blue_E1_224   p322_blue_E1_224.mzML   NA    sample 224      p322 
21  p322_blue_E2_225   p322_blue_E2_225.mzML   NA    sample 225      p322 
22  p322_blue_E3_29    p322_blue_E3_29.mzML    NA    sample 29       p322 
23  p322_blue_E4_303   p322_blue_E4_303.mzML   NA    sample 303      p322 
24  p322_blue_E5_129   p322_blue_E5_129.mzML   NA    sample 129      p322 
25  p322_blue_E6_329   p322_blue_E6_329.mzML   NA    sample 329      p322 
26  p322_blue_E7_87    p322_blue_E7_87.mzML    NA    sample 87       p322 
27  p322_blue_E8_142   p322_blue_E8_142.mzML   NA    sample 142      p322 
28  p323_green_C10_186 p323_green_C10_186.mzML NA    sample 186      p323 
29  p323_green_C11_154 p323_green_C11_154.mzML NA    sample 154      p323 
30  p323_green_C12_287 p323_green_C12_287.mzML NA    sample 287      p323 
⋮   ⋮                  ⋮                       ⋮     ⋮      ⋮        ⋮    
292 p332_green_E6_1116 p332_green_E6_1116.mzML NA    sample 1116     p332 
293 p332_green_E7_1283 p332_green_E7_1283.mzML NA    sample 1283     p332 
294 p332_green_E8_1263 p332_green_E8_1263.mzML NA    sample 1263     p332 
295 p333_red_C10_1291  p333_red_C10_1291.mzML  NA    sample 1291     p333 
296 p333_red_C11_1059  p333_red_C11_1059.mzML  NA    sample 1059     p333 
297 p333_red_C12_1180  p333_red_C12_1180.mzML  NA    sample 1180     p333 
298 p333_red_C5_1043   p333_red_C5_1043.mzML   NA    sample 1043     p333 
299 p333_red_C6_1143   p333_red_C6_1143.mzML   NA    sample 1143     p333 
300 p333_red_C7_1350   p333_red_C7_1350.mzML   NA    sample 1350     p333 
301 p333_red_C8_1292   p333_red_C8_1292.mzML   NA    sample 1292     p333 
302 p333_red_C9_1048   p333_red_C9_1048.mzML   NA    sample 1048     p333 
303 p333_red_D10_1144  p333_red_D10_1144.mzML  NA    sample 1144     p333 
304 p333_red_D11_1223  p333_red_D11_1223.mzML  NA    sample 1223     p333 
305 p333_red_D12_1067  p333_red_D12_1067.mzML  NA    sample 1067     p333 
306 p333_red_D1_1318   p333_red_D1_1318.mzML   NA    sample 1318     p333 
307 p333_red_D2_1183   p333_red_D2_1183.mzML   NA    sample 1183     p333 
308 p333_red_D3_1250   p333_red_D3_1250.mzML   NA    sample 1250     p333 
309 p333_red_D4_1220   p333_red_D4_1220.mzML   NA    sample 1220     p333 
310 p333_red_D5_1212   p333_red_D5_1212.mzML   NA    sample 1212     p333 
311 p333_red_D6_1218   p333_red_D6_1218.mzML   NA    sample 1218     p333 
312 p333_red_D7_1171   p333_red_D7_1171.mzML   NA    sample 1171     p333 
313 p333_red_D8_1222   p333_r

[1]   321 16374

In [3]:
##################
######332v323#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p323'))

subset_data_plate_1

X_p323 <- subset_data_plate_1[, 11:16374]
y_p323 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p323"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid                         Filename                              Batch
1  p323_green_C10_186               p323_green_C10_186.mzML               NA   
2  p323_green_C11_154               p323_green_C11_154.mzML               NA   
3  p323_green_C12_287               p323_green_C12_287.mzML               NA   
4  p323_green_C6_269                p323_green_C6_269.mzML                NA   
5  p323_green_C7_124                p323_green_C7_124.mzML                NA   
6  p323_green_C8_318                p323_green_C8_318.mzML                NA   
7  p323_green_C9_17                 p323_green_C9_17.mzML                 NA   
8  p323_green_D10_228               p323_green_D10_228.mzML               NA   
9  p323_green_D11_338               p323_green_D11_338.mzML               NA   
10 p323_green_D12_101               p323_green_D12_101.mzML               NA   
11 p323_green_D1_325_20240117142545 p323_green_D1_325_20240117142545.mzML NA   
12 p323_green_D2_12                 p323_green_D2_12.mzML                 NA   
13 p323_green_D3_83                 p323_green_D3_83.mzML                 NA   
14 p323_green_D4_285                p323_green_D4_285.mzML                NA   
15 p323_green_D5_286                p323_green_D5_286.mzML                NA   
16 p323_green_D6_341                p323_green_D6_341.mzML                NA   
17 p323_green_D7_298                p323_green_D7_298.mzML                NA   
18 p323_green_D8_131                p323_green_D8_131.mzML                NA   
19 p323_green_E1_189_20240117123944 p323_green_E1_189_20240117123944.mzML NA   
20 p323_green_E2_140                p323_green_E2_140.mzML                NA   
21 p323_green_E3_181                p323_green_E3_181.mzML                NA   
22 p323_green_E4_214                p323_green_E4_214.mzML                NA   
23 p323_green_E5_15                 p323_green_E5_15.mzML                 NA   
24 p323_green_E6_91                 p323_green_E6_91.mzML                 NA   
25 p323_green_E7_259                p323_green_E7_259.mzML                NA   
26 p323_green_E8_239                p323_green_E8_239.mzML                NA   
27 p332_green_C10_1211              p332_green_C10_1211.mzML              NA   
28 p332_green_C11_1178              p332_green_C11_1178.mzML              NA   
29 p332_green_C12_1312              p332_green_C12_1312.mzML              NA   
30 p332_green_C5_1361               p332_green_C5_1361.mzML               NA   
31 p332_green_C6_1293               p332_green_C6_1293.mzML               NA   
32 p332_green_C7_1148               p332_green_C7_1148.mzML               NA   
33 p332_green_C8_1342               p332_green_C8_1342.mzML               NA   
34 p332_green_C9_1041               p332_green_C9_1041.mzML               NA   
35 p332_green_D10_1252              p332_green_D10_1252.mzML              NA   
36 p332_green_D11_1362              p332_green_D11_1362.mzML              NA   
37 p332_green_D12_1126              p332_green_D12_1126.mzML              NA   
38 p332_green_D1_1349               p332_green_D1_1349.mzML               NA   
39 p332_green_D2_1036               p332_green_D2_1036.mzML               NA   
40 p332_green_D3_1108               p332_green_D3_1108.mzML               NA   
41 p332_green_D4_1310               p332_green_D4_1310.mzML               NA   
42 p332_green_D5_1311               p332_green_D5_1311.mzML               NA   
43 p332_green_D6_1365               p332_green_D6_1365.mzML               NA   
44 p332_green_D7_1323               p332_green_D7_1323.mzML               NA   
45 p332_green_D8_1155               p332_green_D8_1155.mzML               NA   
46 p332_green_E1_1214               p332_green_E1_1214.mzML               NA   
47 p332_green_E2_1164               p332_green_E2_1164.mzML               NA   
48 p332_green_E3_1206               p332_green_E3_1206.mzML               NA   
49 p332_green_E4_1238               p332_green_E4_1238.mzML               NA   
50 

In [4]:
#Run Boruta for p322vTarget plate group
boruta_p323 <- Boruta(X_p323, y_p323, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p323, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p323)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_7_LPS_Chateomium_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +47 secs: 

 confirmed 2 attributes: X186.0587_2.65_9814, X336.1669_3.15_5920;

 rejected 16268 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16263 more;

 still have 94 attributes left.


 2

In [5]:
##################
######329v326#####
##################

#Further subsets to only include groups of interest.
subset_data_plate_2 <- subset_data %>%
  filter(plate %in% c('p329', 'p326'))

subset_data_plate_2

X_p326 <- subset_data_plate_2[, 11:16374]
y_p326 <- as.factor(subset(subset_data_plate_2, plate %in% c("p329", "p326"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p326_green_C10_528  p326_green_C10_528.mzML  NA    sample 528      p326 
2  p326_green_C11_495  p326_green_C11_495.mzML  NA    sample 495      p326 
3  p326_green_C12_629  p326_green_C12_629.mzML  NA    sample 629      p326 
4  p326_green_C5_678   p326_green_C5_678.mzML   NA    sample 678      p326 
5  p326_green_C6_611   p326_green_C6_611.mzML   NA    sample 611      p326 
6  p326_green_C7_465   p326_green_C7_465.mzML   NA    sample 465      p326 
7  p326_green_C8_659   p326_green_C8_659.mzML   NA    sample 659      p326 
8  p326_green_C9_358   p326_green_C9_358.mzML   NA    sample 358      p326 
9  p326_green_D10_569  p326_green_D10_569.mzML  NA    sample 569      p326 
10 p326_green_D11_679  p326_green_D11_679.mzML  NA    sample 679      p326 
11 p326_green_D12_443  p326_green_D12_443.mzML  NA    sample 443      p326 
12 p326_green_D1_666   p326_green_D1_666.mzML   NA    sample 666      p326 
13 p326_green_D2_353   p326_green_D2_353.mzML   NA    sample 353      p326 
14 p326_green_D3_425   p326_green_D3_425.mzML   NA    sample 425      p326 
15 p326_green_D4_627   p326_green_D4_627.mzML   NA    sample 627      p326 
16 p326_green_D5_628   p326_green_D5_628.mzML   NA    sample 628      p326 
17 p326_green_D6_682   p326_green_D6_682.mzML   NA    sample 682      p326 
18 p326_green_D7_640   p326_green_D7_640.mzML   NA    sample 640      p326 
19 p326_green_D8_472   p326_green_D8_472.mzML   NA    sample 472      p326 
20 p326_green_E1_531   p326_green_E1_531.mzML   NA    sample 531      p326 
21 p326_green_E2_481   p326_green_E2_481.mzML   NA    sample 481      p326 
22 p326_green_E3_523   p326_green_E3_523.mzML   NA    sample 523      p326 
23 p326_green_E4_555   p326_green_E4_555.mzML   NA    sample 555      p326 
24 p326_green_E5_356   p326_green_E5_356.mzML   NA    sample 356      p326 
25 p326_green_E6_433   p326_green_E6_433.mzML   NA    sample 433      p326 
26 p326_green_E7_600   p326_green_E7_600.mzML   NA    sample 600      p326 
27 p329_green_C10_869  p329_green_C10_869.mzML  NA    sample 869      p329 
28 p329_green_C11_837  p329_green_C11_837.mzML  NA    sample 837      p329 
29 p329_green_C12_970  p329_green_C12_970.mzML  NA    sample 970      p329 
30 p329_green_C5_1020  p329_green_C5_1020.mzML  NA    sample 1020     p329 
31 p329_green_C6_952   p329_green_C6_952.mzML   NA    sample 952      p329 
32 p329_green_C7_807   p329_green_C7_807.mzML   NA    sample 807      p329 
33 p329_green_C8_1000  p329_green_C8_1000.mzML  NA    sample 1000     p329 
34 p329_green_C9_699   p329_green_C9_699.mzML   NA    sample 699      p329 
35 p329_green_D10_911  p329_green_D10_911.mzML  NA    sample 911      p329 
36 p329_green_D11_1021 p329_green_D11_1021.mzML NA    sample 1021     p329 
37 p329_green_D12_784  p329_green_D12_784.mzML  NA    sample 784      p329 
38 p329_green_D1_1008  p329_green_D1_1008.mzML  NA    sample 1008     p329 
39 p329_green_D2_694   p329_green_D2_694.mzML   NA    sample 694      p329 
40 p329_green_D3_766   p329_green_D3_766.mzML   NA    sample 766      p329 
41 p329_green_D4_968   p329_green_D4_968.mzML   NA    sample 968      p329 
42 p329_green_D5_969   p329_green_D5_969.mzML   NA    sample 969      p329 
43 p329_green_D6_1024  p329_green_D6_1024.mzML  NA    sample 1024     p329 
44 p329_green_D7_981   p329_green_D7_981.mzML   NA    sample 981      p329 
45 p329_green_D8_814   p329_green_D8_814.mzML   NA    sample 814      p329 
46 p329_green_E1_872   p329_green_E1_872.mzML   NA    sample 872      p329 
47 p329_green_E2_823   p329_green_E2_823.mzML   NA    sample 823      p329 
48 p329_green_E3_864   p329_green_E3_864.mzML   NA    sample 864      p329 
49 p329_green_E4_896   p329_green_E4_896.mzML   NA    sample 896      p329 
50 p329_green_E5_697   p329_green_E5_697.mzML   NA    sample 697      p329 
51 p329_green_E6_774   p329_green_E6_774.mzML   NA    sample 774      p329 
52 p329_green_E7_942   p329_green_E7_942.mzML   NA 

In [6]:
#Run Boruta for p3XXvTarget plate group
boruta_p326 <- Boruta(X_p326, y_p326, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p326, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p326)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_LPS_Chateomium_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +40 secs: 

 rejected 16342 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16337 more;

 still have 22 attributes left.


 22. run of importance source...

 23. run of importance source...

 2

In [7]:
##################
######332v329#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p329'))

subset_data_plate_1

X_p329 <- subset_data_plate_1[, 11:16374]
y_p329 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p329"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p329_green_C10_869  p329_green_C10_869.mzML  NA    sample 869      p329 
2  p329_green_C11_837  p329_green_C11_837.mzML  NA    sample 837      p329 
3  p329_green_C12_970  p329_green_C12_970.mzML  NA    sample 970      p329 
4  p329_green_C5_1020  p329_green_C5_1020.mzML  NA    sample 1020     p329 
5  p329_green_C6_952   p329_green_C6_952.mzML   NA    sample 952      p329 
6  p329_green_C7_807   p329_green_C7_807.mzML   NA    sample 807      p329 
7  p329_green_C8_1000  p329_green_C8_1000.mzML  NA    sample 1000     p329 
8  p329_green_C9_699   p329_green_C9_699.mzML   NA    sample 699      p329 
9  p329_green_D10_911  p329_green_D10_911.mzML  NA    sample 911      p329 
10 p329_green_D11_1021 p329_green_D11_1021.mzML NA    sample 1021     p329 
11 p329_green_D12_784  p329_green_D12_784.mzML  NA    sample 784      p329 
12 p329_green_D1_1008  p329_green_D1_1008.mzML  NA    sample 1008     p329 
13 p329_green_D2_694   p329_green_D2_694.mzML   NA    sample 694      p329 
14 p329_green_D3_766   p329_green_D3_766.mzML   NA    sample 766      p329 
15 p329_green_D4_968   p329_green_D4_968.mzML   NA    sample 968      p329 
16 p329_green_D5_969   p329_green_D5_969.mzML   NA    sample 969      p329 
17 p329_green_D6_1024  p329_green_D6_1024.mzML  NA    sample 1024     p329 
18 p329_green_D7_981   p329_green_D7_981.mzML   NA    sample 981      p329 
19 p329_green_D8_814   p329_green_D8_814.mzML   NA    sample 814      p329 
20 p329_green_E1_872   p329_green_E1_872.mzML   NA    sample 872      p329 
21 p329_green_E2_823   p329_green_E2_823.mzML   NA    sample 823      p329 
22 p329_green_E3_864   p329_green_E3_864.mzML   NA    sample 864      p329 
23 p329_green_E4_896   p329_green_E4_896.mzML   NA    sample 896      p329 
24 p329_green_E5_697   p329_green_E5_697.mzML   NA    sample 697      p329 
25 p329_green_E6_774   p329_green_E6_774.mzML   NA    sample 774      p329 
26 p329_green_E7_942   p329_green_E7_942.mzML   NA    sample 942      p329 
27 p329_green_E8_922   p329_green_E8_922.mzML   NA    sample 922      p329 
28 p332_green_C10_1211 p332_green_C10_1211.mzML NA    sample 1211     p332 
29 p332_green_C11_1178 p332_green_C11_1178.mzML NA    sample 1178     p332 
30 p332_green_C12_1312 p332_green_C12_1312.mzML NA    sample 1312     p332 
31 p332_green_C5_1361  p332_green_C5_1361.mzML  NA    sample 1361     p332 
32 p332_green_C6_1293  p332_green_C6_1293.mzML  NA    sample 1293     p332 
33 p332_green_C7_1148  p332_green_C7_1148.mzML  NA    sample 1148     p332 
34 p332_green_C8_1342  p332_green_C8_1342.mzML  NA    sample 1342     p332 
35 p332_green_C9_1041  p332_green_C9_1041.mzML  NA    sample 1041     p332 
36 p332_green_D10_1252 p332_green_D10_1252.mzML NA    sample 1252     p332 
37 p332_green_D11_1362 p332_green_D11_1362.mzML NA    sample 1362     p332 
38 p332_green_D12_1126 p332_green_D12_1126.mzML NA    sample 1126     p332 
39 p332_green_D1_1349  p332_green_D1_1349.mzML  NA    sample 1349     p332 
40 p332_green_D2_1036  p332_green_D2_1036.mzML  NA    sample 1036     p332 
41 p332_green_D3_1108  p332_green_D3_1108.mzML  NA    sample 1108     p332 
42 p332_green_D4_1310  p332_green_D4_1310.mzML  NA    sample 1310     p332 
43 p332_green_D5_1311  p332_green_D5_1311.mzML  NA    sample 1311     p332 
44 p332_green_D6_1365  p332_green_D6_1365.mzML  NA    sample 1365     p332 
45 p332_green_D7_1323  p332_green_D7_1323.mzML  NA    sample 1323     p332 
46 p332_green_D8_1155  p332_green_D8_1155.mzML  NA    sample 1155     p332 
47 p332_green_E1_1214  p332_green_E1_1214.mzML  NA    sample 1214     p332 
48 p332_green_E2_1164  p332_green_E2_1164.mzML  NA    sample 1164     p332 
49 p332_green_E3_1206  p332_green_E3_1206.mzML  NA    sample 1206     p332 
50 p332_green_E4_1238  p332_green_E4_1238.mzML  NA    sample 1238     p332 
51 p332_green_E5_1039  p332_green_E5_1039.mzML  NA    sample 1039     p332 
52 p332_green_E6_1116  p332_green_E6_1116.mzML  NA 

In [8]:
#Run Boruta for p322vTarget plate group
boruta_p329 <- Boruta(X_p329, y_p329, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p329, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p329)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_No_LPS_Chateomium.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +39 secs: 

 confirmed 3 attributes: X186.0587_2.65_9814, X257.0245_0.25_3146, X358.9965_1.17_3536;

 rejected 16252 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16247 more;

 still have 109